In [4]:
import pandas as pd
import numpy as np
#import mathplotlib.pyplot as plt

In [6]:
r = 'data/'
check = pd.read_csv(r + 'check.csv', sep=';', encoding='cp1252')
hp = pd.read_csv(r + 'hp.csv', sep=';', encoding='cp1252')
coord = pd.read_csv(r + 'coord.csv', sep=';', encoding='cp1252')
ght = pd.read_csv(r + 'ght.csv', sep=';', encoding='cp1252')
ssr = pd.read_csv(r + 'ssr.csv', sep=';', encoding='cp1252')

In [7]:
#check encoding
with open("data/coord.csv") as f:
    print(f.encoding)

cp1252


## TOCHECK

- nb d'établissements. Nom à uniformiser
- nb GHT
- y a-t-il toutes les coordonnées ?
- pb : transferts entre [0-10]
- rename the columns



In [8]:
#check nb établissements
check

,Finess,Raison sociale,Catégorie,Provenance,Nb de transferts
0,710978248,CH AUTUN SITE LATOUCHE,CH,CH AUTUN SITE PARPAS,155
1,580780187,CLINIQUE DU MORVAN SA,Privé,CH AUTUN SITE PARPAS,48
2,580780187,CLINIQUE DU MORVAN SA,Privé,CH DECIZE,34
3,580780187,CLINIQUE DU MORVAN SA,Privé,HÔPITAL PIERRE BEREGOVOY,16
4,710002288,SSR MARGUERITE BOUCICAUT,Privé,CH AUTUN SITE PARPAS,21
...,...,...,...,...,...
141,890975543,CH JOIGNY,CH,CH JOIGNY,149
142,100007285,CRF INSTITUT ASCLEPIADE,Privé,CH SENS,37
143,100010362,GCS PLAT AVAL TERR CHAMP SUD CRF PAST,Privé,CH SENS,19
144,450000526,CMPR L'A.D.A.P.T. LOIRET,PSPH/EBNL,CH SENS,14


### check etab missing

In [9]:
check['Raison sociale'].drop_duplicates().count()
# => 74 établissements au total
hp['Etablissement'].drop_duplicates().count()
#=> 49 hp
ssr['Etablissement SSR '].drop_duplicates().count()
# => 82 ssr
# 82 + 49 = 131 
#- intersect = 111

82

In [10]:
bourgogne_etab = check['Raison sociale'].drop_duplicates()
hp_etab = hp['Etablissement'].drop_duplicates()
ssr_etab = ssr['Etablissement SSR '].drop_duplicates()
coord_etab = coord['Etablissement SSR '].drop_duplicates()

In [11]:
bourgogne_etab.name = 'etab'
hp_etab.name = 'etab'
ssr_etab.name = 'etab'
coord_etab.name = 'etab'

In [12]:
#both in hp and ssr
both_in_hp_ssr = pd.merge(hp_etab, ssr_etab, how='inner', left_on=hp_etab.name, right_on=ssr_etab.name)
both_in_hp_ssr.shape

(20, 1)

In [13]:
both_in_hp_ssr

,etab
0,CH WILLIAM MOREY CHALON SUR SAONE
1,CH BRESSE LOUHANNAISE
2,CH JEAN BOUVERI
3,CH AUXERRE
4,CH DU TONNERROIS
5,CH DE LANGRES
6,CH HCO SITE DE CHATILLON SUR SEINE
7,CH HCO SITE DE MONTBARD
8,CH D'IS-SUR-TILLE
9,CH ALIGRE BOURBON LANCY


In [14]:
hp_ssr = pd.concat([hp_etab, ssr_etab], axis=0)

In [15]:
hp_ssr = hp_ssr.drop_duplicates()

In [16]:
hp_ssr.shape

(111,)

In [17]:
hp_ssr

0                   CH AUTUN SITE PARPAS
36     CH WILLIAM MOREY CHALON SUR SAONE
131                CH BRESSE LOUHANNAISE
143                      CH JEAN BOUVERI
189                            HL CHAGNY
                     ...                
398             CMPR L'A.D.A.P.T. LOIRET
401              CENTRE DE CONVALESCENCE
404             CENTRE SSR DU CHALONNAIS
405                SA MAISON DE JOUVENCE
413                      CH DE LA GUICHE
Name: etab, Length: 111, dtype: object

In [18]:
#Etablissements dans la liste 'Bourgogne' mais pas dans les hp ni les ssr
bourgogne_etab[bourgogne_etab.isin(hp_ssr) == False]

18                        KORIAN LA BRESSANE
74                  CH REG UNIVERSITAIRE DIJ
82                              CH D'AUXONNE
96             CENTRE MÉDICAL LA GRANGE/MONT
99                   SARL JOUVENCE NUTRITION
102                       CH DE BAR SUR AUBE
103                     HÔPITAL DE JOINVILLE
105                CLINIQUE DE LA COMPASSION
118                            CH DE BEAUJEU
119    CH PAYS CHAROLAIS BRIONNAIS CHAROLLES
122                          CR LES ARBELLES
133              INSTITUT MEDICAL DE SOLOGNE
134                    MOYEN SEJOUR PIGNELIN
136                     MOYEN SEJOUR CH SENS
143    GCS PLAT AVAL TERR CHAMP SUD CRF PAST
145                 GCS PATCS CRRF PASTEUR 2
Name: etab, dtype: object

In [19]:
#Etablissement dans les onglets hp et ssr mais pas dans coord
hp_ssr[hp_ssr.isin(coord_etab) == False] #45

189                                  HL CHAGNY
200                                 CHS SEVREY
201                                CHS AUXERRE
293                                 CH AVALLON
319                           CH DU TONNERROIS
339                                 CH CLAMECY
364              \tCH REG UNIVERSITAIRE DIJON 
396                   \tCH LA CHARTREUSE DIJON
400             \tHÔPITAL LE BOCAGE CHRU DIJON
529                           \tCH DE CHAUMONT
598                  CH DE BOURBONNE-LES-BAINS
621                    CH HCO SITE DE MONTBARD
640                     CH HCO SITE DE SAULIEU
659                         CH ROBERT MORLEVAT
716                \tHOSPICES CIVILS DE BEAUNE
896             CH DU PAYS CHAROLAIS BRIONNAIS
997     CH PAYS CHAROLAIS BRIONNAIS LA CLAYETT
1205                                   CH SENS
1292                                 CH JOIGNY
1386               HÔPITAL PRIVE SAINTE MARIE 
1441             POLYCLINIQUE DU VAL DE SAONE 
1492         

In [166]:
#uniformisation des noms d'établissements
#TODO

### uniformisation des noms des variables 

In [20]:
hp.columns

Index(['Nom_GHT', 'Etablissement', 'Rang', 'Code', 'Groupe Plannification',
       'Nb séjour / Séances', 'Nb de jours', 'Age moyen', '% Age>75',
       '% lourd',
       'Transferts SSR (Nombre de séjours chainés et tranférés vers un établissement SSR) ',
       'Etablissement SSR de transfert ', 'Catégorie ', 'Nb de transferts',
       'is_in_GHT', 'transfer_interne'],
      dtype='object')

In [21]:
hp.columns = [
    'nom_ght', 
    'etablissement',
    'rang',
    'code',
    'groupe_plannification',
    'nb_sejour_par_seance',
    'nb_jour',
    'age_moyen',
    'pourc_age_sup_75',
    'pourc_lourd',
    'transferts_ssr',
    'etablissement_ssr_de_transfert',
    'categorie',
    'nb_transferts',
    'is_in_ght',
    'transfert_interne'
]

In [22]:
ssr.columns

Index(['Nom_GHT', 'Etablissement SSR ', 'Catégorie ', 'Code ',
       'Groupe planification ', 'Provenance ', 'Effectif ',
       'Total par établissement (Effecitifs >10)', 'Total (Effecitifs >10)',
       'is_in_GHT', 'Unnamed: 10'],
      dtype='object')

In [23]:
ssr.columns = [
    'nom_ght',
    'etablissement_ssr',
    'categorie',
    'code',
    'groupe_planification',
    'provenance',
    'effectif_entrant',
    'todelete',
    'todelete2',
    'is_in_ght',
    'todelete3'
]

In [24]:
ssr = ssr.drop(['todelete', 'todelete2', 'todelete3'], axis=1)

In [25]:
ssr['is_in_ght'] = ssr['is_in_ght'].fillna(0)

In [26]:
check.columns 

Index(['Finess', 'Raison sociale', 'Catégorie', 'Provenance ',
       'Nb de transferts'],
      dtype='object')

In [27]:
check.columns = ['finess', 'etablissement', 'categorie', 'provenance', 'nb_transferts']

In [28]:
check

,finess,etablissement,categorie,provenance,nb_transferts
0,710978248,CH AUTUN SITE LATOUCHE,CH,CH AUTUN SITE PARPAS,155
1,580780187,CLINIQUE DU MORVAN SA,Privé,CH AUTUN SITE PARPAS,48
2,580780187,CLINIQUE DU MORVAN SA,Privé,CH DECIZE,34
3,580780187,CLINIQUE DU MORVAN SA,Privé,HÔPITAL PIERRE BEREGOVOY,16
4,710002288,SSR MARGUERITE BOUCICAUT,Privé,CH AUTUN SITE PARPAS,21
...,...,...,...,...,...
141,890975543,CH JOIGNY,CH,CH JOIGNY,149
142,100007285,CRF INSTITUT ASCLEPIADE,Privé,CH SENS,37
143,100010362,GCS PLAT AVAL TERR CHAMP SUD CRF PAST,Privé,CH SENS,19
144,450000526,CMPR L'A.D.A.P.T. LOIRET,PSPH/EBNL,CH SENS,14


In [29]:
coord.columns

Index(['Etablissement SSR ', 'Adresse', 'Latitude ', 'Longitude'], dtype='object')

In [30]:
coord.columns = ['etablissement', 'adresse', 'latitude', 'longitude']

In [31]:
coord

,etablissement,adresse,latitude,longitude
0,AIHP CENTRE ARMANCON,18 Bis Rue Pierre Semard,"47,95981","3,53041"
1,C.R.B LES ROSIERS,45 Bd Henri Bazin,"47,30041","5,019"
2,CENTRE COLBERT MEDECINE ET SSR,4 Rue Étienne Litaud,"46,99469","3,1597"
3,CENTRE DE CONVALESCENCE,67 Rte d'Ahuy,"47,34583","5,03471"
4,CENTRE MÉDICAL DE LA VENERIE,Lieu Dit La Vénerie,"47,2668","3,2896"
...,...,...,...,...
73,POLYCLINIQUE DU VAL DE SAONE,44 Rue Ambroise Paré,"46,3173","4,81178"
74,POLYCLINIQUE STE MARGUERITE,5 Av. de la Font Sainte-Marguerite,"47,81213","3,56478"
75,SA MAISON JOUVENCE,20 Rue des Alisiers,"47,40975","5,00968"
76,SSR EDITH CAVELL,27 Av. Françoise Giroud,"47,36094","5,05098"


In [32]:
ght.columns

Index(['GHT', 'Code', 'Groupe Plannification', 'Nb séjour / Séances',
       'Nb de jours', '/', '/2', '/ ',
       'Transferts SSR (Nombre de séjours chainés et tranférés vers un établissement SSR) ',
       'Pourcentage '],
      dtype='object')

In [33]:
ght.columns = [
    'nom_ght',
    'code',
    'groupe_plannification',
    'nb_sejour_par_seance',
    'nb_jour',
    'todelete1',
    'todelete2',
    'todelete3',
    'transferts_ssr',
    'pourcentage'
]

In [34]:
ght.columns

Index(['nom_ght', 'code', 'groupe_plannification', 'nb_sejour_par_seance',
       'nb_jour', 'todelete1', 'todelete2', 'todelete3', 'transferts_ssr',
       'pourcentage'],
      dtype='object')

In [35]:
ght = ght.drop(['todelete1', 'todelete2', 'todelete3'], axis=1)

In [36]:
ght

,nom_ght,code,groupe_plannification,nb_sejour_par_seance,nb_jour,transferts_ssr,pourcentage
0,GHT SÂONE-ET-LOIRE BRESSE MORVAN,X03,Neurologie,2828,17255,394,"0,139321075"
1,GHT SÂONE-ET-LOIRE BRESSE MORVAN,X08,Pneumologie,4237,31189,391,"0,092282275"
2,GHT SÂONE-ET-LOIRE BRESSE MORVAN,C09,"Chir. majeure de l'app. Locomoteur:\n hanche, ...",637,4825,272,"0,42700157"
3,GHT SÂONE-ET-LOIRE BRESSE MORVAN,X07,Affections Cardio-vasculaires,3266,18585,196,"0,060012247"
4,GHT SÂONE-ET-LOIRE BRESSE MORVAN,X06,Rhumatologie,1279,9071,140,"0,109460516"
...,...,...,...,...,...,...,...
371,GHT Nord-Yonne,O01,Accouchement par voie basse,924,3574,0 à 10,"0,010822511"
372,GHT Nord-Yonne,S01,Dialyse,7278,0,NaN,/
373,GHT Nord-Yonne,S02,Chimiothérapie pour tumeurs,3537,0,NaN,/
374,GHT Nord-Yonne,S03,Chimiothérapie pour aff. Non tumorale,658,0,NaN,/


In [37]:
check.columns

Index(['finess', 'etablissement', 'categorie', 'provenance', 'nb_transferts'], dtype='object')

### uniformisation des noms (ght et etablissement)

In [183]:
hp['nom_ght'] = hp['nom_ght'].str.upper()
hp['nom_ght'] = hp['nom_ght'].str.lstrip()
hp['nom_ght'] = hp['nom_ght'].str.rstrip()
hp['nom_ght'] = hp['nom_ght'].str.replace('\t', '')

In [268]:
hp['etablissement'] = hp['etablissement'].str.upper()
hp['etablissement'] = hp['etablissement'].str.lstrip()
hp['etablissement'] = hp['etablissement'].str.rstrip()
hp['etablissement'] = hp['etablissement'].str.replace('\t', '')
hp.loc[hp.etablissement == 'CH PAYS CHAROLAIS BRIONNAIS LA CLAYETT', 'etablissement'] = 'CH PAYS CHAROLAIS BRIONNAIS LA CLAYETTE'

In [184]:
hp['etablissement_ssr_de_transfert'] = hp['etablissement_ssr_de_transfert'].str.upper()
hp['etablissement_ssr_de_transfert'] = hp['etablissement_ssr_de_transfert'].str.lstrip()
hp['etablissement_ssr_de_transfert'] = hp['etablissement_ssr_de_transfert'].str.rstrip()
hp['etablissement_ssr_de_transfert'] = hp['etablissement_ssr_de_transfert'].str.replace('\t', '')

In [185]:
ssr['nom_ght'] = ssr['nom_ght'].str.upper()
ssr['nom_ght'] = ssr['nom_ght'].str.lstrip()
ssr['nom_ght'] = ssr['nom_ght'].str.rstrip()
ssr['nom_ght'] = ssr['nom_ght'].str.replace('\t', '')

In [186]:
ssr['provenance'] = ssr['provenance'].str.upper()
ssr['provenance'] = ssr['provenance'].str.lstrip()
ssr['provenance'] = ssr['provenance'].str.rstrip()
ssr['provenance'] = ssr['provenance'].str.replace('\t', '')

In [187]:
ssr['etablissement_ssr'] = ssr['etablissement_ssr'].str.upper()
ssr['etablissement_ssr'] = ssr['etablissement_ssr'].str.lstrip()
ssr['etablissement_ssr'] = ssr['etablissement_ssr'].str.rstrip()
ssr['etablissement_ssr'] = ssr['etablissement_ssr'].str.replace('\t', '')

In [188]:
ght['nom_ght'] = ght['nom_ght'].str.upper()
ght['nom_ght'] = ght['nom_ght'].str.lstrip()
ght['nom_ght'] = ght['nom_ght'].str.rstrip()
ght['nom_ght'] = ght['nom_ght'].str.replace('\t', '')

In [189]:
check['etablissement'] = check['etablissement'].str.upper()
check['etablissement'] = check['etablissement'].str.lstrip()
check['etablissement'] = check['etablissement'].str.rstrip()
check['etablissement'] = check['etablissement'].str.replace('\t', '')

In [190]:
check['provenance'] = check['provenance'].str.upper()
check['provenance'] = check['provenance'].str.lstrip()
check['provenance'] = check['provenance'].str.rstrip()
check['provenance'] = check['provenance'].str.replace('\t', '')

In [191]:
check['categorie'] = check['categorie'].str.upper()
check['categorie'] = check['categorie'].str.lstrip()
check['categorie'] = check['categorie'].str.rstrip()
check['categorie'] = check['categorie'].str.replace('\t', '')

In [192]:
coord['etablissement'] = coord['etablissement'].str.upper()
coord['etablissement'] = coord['etablissement'].str.lstrip()
coord['etablissement'] = coord['etablissement'].str.rstrip()
coord['etablissement'] = coord['etablissement'].str.replace('\t', '')

### modification des types (after)

In [193]:
coord['latitude'] = coord['latitude'].str.replace(',', '.')
coord['longitude'] = coord['longitude'].str.replace(',', '.')

AttributeError: Can only use .str accessor with string values!

In [73]:
coord['latitude'] = coord['latitude'].astype('float64')
coord['longitude'] = coord['longitude'].astype('float64')

In [74]:
coord.dtypes

etablissement     object
adresse           object
latitude         float64
longitude        float64
dtype: object

### sauvegarde des df

In [270]:
r = './data/new_data/'
coord.to_csv(r + 'coord.csv')
hp.to_csv(r + 'hp.csv')
ssr.to_csv(r + 'ssr.csv')
ght.to_csv(r + 'ght.csv')
check.to_csv(r + 'check.csv')

## Back to the check

In [3]:
import pandas as pd
import numpy as np

In [4]:
r = './data/new_data/'
coord = pd.read_csv(r + 'coord.csv', sep=',', index_col=0)
hp = pd.read_csv(r + 'hp.csv', sep=',', index_col=0)
ssr = pd.read_csv(r + 'ssr.csv', sep=',', index_col=0)
ght = pd.read_csv(r + 'ght.csv', sep=',', index_col=0)
check = pd.read_csv(r + 'check.csv', sep=',', index_col=0)

### Vérifions qu'ils y a bien toutes les coordonnées

In [7]:
coord_etab = coord['etablissement'].drop_duplicates()
ssr_concat_hp_etab = pd.concat([hp['etablissement'].drop_duplicates(), ssr['etablissement_ssr'].drop_duplicates()])
ssr_concat_hp_etab = ssr_concat_hp_etab.drop_duplicates()

In [8]:
print(ssr_concat_hp_etab.shape, coord_etab.shape)

(91,) (78,)


In [9]:
#dans les onglets hp et ssr mais pas dans coord ?
missing_coord = ssr_concat_hp_etab[ssr_concat_hp_etab.isin(coord_etab) == False] #47

In [10]:
missing_coord.count()

18

In [277]:
78+19

97

In [13]:
#dans coord mais pas ds hp ni ssr
coord_etab[coord_etab.isin(ssr_concat_hp_etab) == False]

20                            CH DE BEAUJEU
49                          CR LES ARBELLES
57                 GCS PATCS CRRF PASTEUR 2
58    GCS PLAT AVAL TERR CHAMP SUD CRF PAST
72                    MOYEN SEJOUR PIGNELIN
Name: etablissement, dtype: object

In [11]:
missing_coord

319                   CH DU TONNERROIS
396             CH LA CHARTREUSE DIJON
598          CH DE BOURBONNE-LES-BAINS
659                 CH ROBERT MORLEVAT
1386        HÔPITAL PRIVE SAINTE MARIE
1492       POLYCLINIQUE DU PARC DREVON
1537      CLINIQUE MEDICO-CHIRURGICALE
1593                  CLINIQUE DU PARC
1621              CLINIQUE PAUL PIQUET
1669     HOPITAL PRIVE DIJON BOURGOGNE
1737    CENTRE GEORGE-FRANCOIS LECLERC
1784             HOTEL-DIEU DU CREUSOT
100       CENTRE MEDICAL DE LA VENERIE
208              CH LES CHAUNAUX MACON
241                          CH BELANY
273                CRF LES BOURBONNAIS
382                 C.R.B. LES ROSIERS
405              SA MAISON DE JOUVENCE
dtype: object

In [311]:
missing_coord.to_csv('missing_coord.csv', encoding='cp1252')

In [253]:
#hp + ssr = 121, sans l'inter = 92
ssr_concat_hp_etab.drop_duplicates().count()

92

In [279]:
coord_etab.count() + missing_coord.count() #78 + 22 = 90

#=> 4 qui sont probablement ds coord mais mal orthographié

96

### Vérifions les ght

In [344]:
ght['nom_ght'].drop_duplicates()

0      GHT SÂONE-ET-LOIRE BRESSE MORVAN
62         GHT SUD YONNE HAUT-NIVERNAIS
121         GHT CÔTE D'OR - HAUTE MARNE
197                   GHT SUD CÔTE-D'OR
260                    GHT DE LA NIEVRE
321                      GHT NORD-YONNE
Name: nom_ght, dtype: object

In [337]:
hp['nom_ght'].drop_duplicates()

0       GHT SÂONE-ET-LOIRE BRESSE MORVAN
201         GHT SUD YONNE HAUT-NIVERNAIS
364          GHT CÔTE D'OR - HAUTE MARNE
716                    GHT SUD CÔTE-D'OR
782            GHT BOURGOGNE MÉRIDIONALE
1009                    GHT DE LA NIÈVRE
1205                     GHT  NORD-YONNE
1320                                NONE
Name: nom_ght, dtype: object

In [346]:
#hp.loc[hp.nom_ght == '', 'nom_ght'] = ''
hp.loc[hp.nom_ght == 'GHT SAÔNEETLOIRE BRESSE MORVAN', 'nom_ght'] = 'GHT SÂONE-ET-LOIRE BRESSE MORVAN'
ssr.loc[ssr.nom_ght == 'GHT SAÔNEETLOIRE BRESSE MORVAN', 'nom_ght'] = 'GHT SÂONE-ET-LOIRE BRESSE MORVAN'

In [347]:
hp.to_csv(r + 'hp.csv')
ssr.to_csv(r + 'ssr.csv')

### Check transferts GHT (et donc sum établissement rattachés aux GHT) => transferts_ssr

In [20]:
ght.groupby(['nom_ght'])['transferts_ssr'].sum()

nom_ght
GHT CÔTE D'OR - HAUTE MARNE         614 à 624525464 à 474408 à 418 282 à 292 20518...
GHT DE LA NIEVRE                    275217 à 237173 à 203 148 à 15899 à 12998 à 12...
GHT NORD-YONNE                      24718412412499585751 à 6150 à 60 47393829 à 39...
GHT SUD CÔTE-D'OR                   599343 à 373232 à 262197 à 217 191 à 221172 à ...
GHT SUD YONNE HAUT-NIVERNAIS        24486 à 9614020388 à 982049 à 6920 à 301220 à ...
GHT SÂONE-ET-LOIRE BRESSE MORVAN    39439127219614012810796948263 à 8349 à 6949 à ...
Name: transferts_ssr, dtype: object

In [26]:
ght['transferts_ssr'].drop_duplicates()

0           394
1           391
2           272
3           196
4           140
         ...   
330          47
333    29 à 39 
334     23 à 33
335          17
336          15
Name: transferts_ssr, Length: 132, dtype: object

In [38]:
x = ght['transferts_ssr'].drop_duplicates()

In [42]:
_int = x[x.str.isdigit() == False]

In [50]:
_int[_int.str.find('à') == 3]

10       63 à 83
11       49 à 69
12      49 à 59 
14      32 à 42 
16       27 à 47
19       16 à 36
21       11 à 51
63       86 à 96
66       88 à 98
69       20 à 30
71       20 à 50
76       41 à 61
77      13 à 23 
79       36 à 56
80      68 à 108
81       46 à 66
83      94 à 114
133     99 à 139
134     92 à 122
135    91 à 141 
136     81 à 91 
137    75 à 135 
138     71 à 81 
140    50 à 110 
143     35 à 45 
144     30 à 40 
145     28 à 38 
146      24 à 44
148      18 à 78
150      14 à 74
151     14 à 24 
152      11 à 71
153     11 à 21 
206     95 à 125
207    87 à 117 
208     87 à 117
211     45 à 75 
213     33 à 63 
220     20 à 30 
221      18 à 58
222     12 à 22 
223     11 à 31 
264     99 à 129
265     98 à 128
266      78 à 88
267     76 à 116
268      61 à 71
269     56 à 106
270      56 à 66
271     46 à 56 
272      38 à 68
273      29 à 69
274      24 à 54
278      13 à 33
328      51 à 61
329     50 à 60 
333     29 à 39 
334      23 à 33
Name: transfer

In [59]:
tmp = pd.Series(['0 à 50', '197 à 217', '63 à 83', np.nan, '#REF', '67'])
tmp

0       0 à 50
1    197 à 217
2      63 à 83
3          NaN
4         #REF
5           67
dtype: object

In [60]:
tmp.str.extract(pat='(\d*) ?à ?(\d*)')

,0,1
0,0,50
1,197,217
2,63,83
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN


In [ ]:
#REF!

In [62]:
ght.loc[ght.transferts_ssr.isna()]

,nom_ght,code,groupe_plannification,nb_sejour_par_seance,nb_jour,transferts_ssr,pourcentage
58,GHT SÂONE-ET-LOIRE BRESSE MORVAN,S01,Dialyse,13380,0,NaN,/
59,GHT SÂONE-ET-LOIRE BRESSE MORVAN,S02,Chimiothérapie pour tumeurs,9646,0,NaN,/
60,GHT SÂONE-ET-LOIRE BRESSE MORVAN,S03,Chimiothérapie pour aff. Non tumorale,2314,0,NaN,/
61,GHT SÂONE-ET-LOIRE BRESSE MORVAN,S05,Transfusions,1389,0,NaN,/
116,GHT SUD YONNE HAUT-NIVERNAIS,S01,Dialyse,7642,0,NaN,/
117,GHT SUD YONNE HAUT-NIVERNAIS,S02,Chimiothérapie pour tumeurs,6557,0,NaN,/
118,GHT SUD YONNE HAUT-NIVERNAIS,S03,Chimiothérapie pour aff. Non tumorale,1046,#REF!,NaN,/
119,GHT SUD YONNE HAUT-NIVERNAIS,S05,Transfusions,575,0,NaN,/
120,GHT SUD YONNE HAUT-NIVERNAIS,S06,Aphérèse,212,0,NaN,NaN
192,GHT CÔTE D'OR - HAUTE MARNE,S01,Dialyse,16268,0,NaN,/


In [63]:
ssr

,nom_ght,etablissement_ssr,categorie,code,groupe_planification,provenance,effectif_entrant,is_in_ght
0,GHT SÂONE-ET-LOIRE BRESSE MORVAN,CH AUTUN SITE LATOUCHE,CH,X08,Pneumologie,CH AUTUN SITE PARPAS,30,1.0
1,GHT SÂONE-ET-LOIRE BRESSE MORVAN,CH AUTUN SITE LATOUCHE,CH,X03,Neurologie,CH AUTUN SITE PARPAS,30,1.0
2,GHT SÂONE-ET-LOIRE BRESSE MORVAN,CH AUTUN SITE LATOUCHE,CH,X07,Affections Cardio-vasculaires,CH AUTUN SITE PARPAS,16,1.0
3,GHT SÂONE-ET-LOIRE BRESSE MORVAN,CH AUTUN SITE LATOUCHE,CH,X06,Rhumatologie,CH AUTUN SITE PARPAS,12,1.0
4,GHT SÂONE-ET-LOIRE BRESSE MORVAN,CENTRE ORTHOPEDIQUE MEDICO-CHIR,Privé,C09,"Chir. majeure de l'app. Locomoteur:\n hanche, ...",CH WILLIAM MOREY CHALON SUR SAONE,78,1.0
...,...,...,...,...,...,...,...,...
409,NONE,HOTEL-DIEU DU CREUSOT,PSPH/EBNL,X06,Rhumatologie,HOTEL-DIEU DU CREUSOT,19,0.0
410,NONE,HOTEL-DIEU DU CREUSOT,PSPH/EBNL,X07,Affection cardio-vasculaire,HOTEL-DIEU DU CREUSOT,19,0.0
411,NONE,HOTEL-DIEU DU CREUSOT,PSPH/EBNL,X03,Neurologie médicale,HOTEL-DIEU DU CREUSOT,11,0.0
412,NONE,CH JEAN BOUVERI,CH,C09,"Chir. majeure de l'app. Locomoteur:\n hanche, ...",HOTEL-DIEU DU CREUSOT,21,0.0


ssr.groupby('nom_ght')

In [70]:
ssr.groupby('nom_ght')['effectif_entrant'].sum()

nom_ght
GHT BOURGOGNE MÉRIDIONALE           1887
GHT CÔTE D'OR - HAUTE MARNE         2644
GHT DE LA NIÈVRE                     953
GHT SUD CÔTE-D'OR                    318
GHT SUD YONNE HAUT-NIVERNAIS        1286
GHT SÂONE-ET-LOIRE BRESSE MORVAN    1491
GHT  NORD-YONNE                      813
NONE                                2233
Name: effectif_entrant, dtype: int64

In [75]:
check

,finess,etablissement,categorie,provenance,nb_transferts
0,710978248,CH AUTUN SITE LATOUCHE,CH,CH AUTUN SITE PARPAS,155
1,580780187,CLINIQUE DU MORVAN SA,PRIVÉ,CH AUTUN SITE PARPAS,48
2,580780187,CLINIQUE DU MORVAN SA,PRIVÉ,CH DECIZE,34
3,580780187,CLINIQUE DU MORVAN SA,PRIVÉ,HÔPITAL PIERRE BEREGOVOY,16
4,710002288,SSR MARGUERITE BOUCICAUT,PRIVÉ,CH AUTUN SITE PARPAS,21
...,...,...,...,...,...
141,890975543,CH JOIGNY,CH,CH JOIGNY,149
142,100007285,CRF INSTITUT ASCLEPIADE,PRIVÉ,CH SENS,37
143,100010362,GCS PLAT AVAL TERR CHAMP SUD CRF PAST,PRIVÉ,CH SENS,19
144,450000526,CMPR L'A.D.A.P.T. LOIRET,PSPH/EBNL,CH SENS,14


In [107]:
check_etab = pd.concat([check['etablissement'].drop_duplicates(), check['provenance'].drop_duplicates()])
check_etab = check_etab.drop_duplicates()

coord

In [114]:
check_etab.count()

83

In [116]:
x = check_etab[check_etab.isin(coord_etab)==False]

In [118]:
x[74]

'CH REG UNIVERSITAIRE DIJ'

In [120]:
x

48                           CH DU TONNERROIS
74                   CH REG UNIVERSITAIRE DIJ
80                      SA MAISON DE JOUVENCE
89                         C.R.B. LES ROSIERS
96              CENTRE MÉDICAL LA GRANGE/MONT
99                    SARL JOUVENCE NUTRITION
102                        CH DE BAR SUR AUBE
103                      HÔPITAL DE JOINVILLE
105                 CLINIQUE DE LA COMPASSION
106                 CH DE BOURBONNE-LES-BAINS
7                        CHRU LE BOCAGE DIJON
38                           CH LA CHARTREUSE
73                         CH ROBERT MORLEVAT
121    CH PAYS CHAROLAIS BRIONNAIS LA CLAYETT
dtype: object

In [122]:
x[48]

'CH DU TONNERROIS'

In [125]:
coord_etab[28]

'CH DU TONNEROIS'